In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

In [ ]:
# Extract wanted columns from dataframe
df_cur1 = pd.read_csv('20170601_EU.csv')
cur1_col1 = df_cur1.iloc[:, 0:1].values
#cur1_col2 = df_cur1.iloc[1]

df_cur2 = pd.read_csv('20170601_EC.csv')
cur2_col1 = df_cur2.iloc[:, 0:1].values
#cur2_col2 = df_cur2.iloc[1]

In [ ]:
# Sample size parameter
samplesize = 2000

other_ratio, this_ratio, cur1_localmax, cur2_localmax, cur1_localmin, cur2_localmin = ([] for i in range(6))

for i in range(samplesize, len(cur1_col1)-1):
    #cur1_localmax.insert(i-samplesize, float(max(cur1_col1[i-samplesize:i-1])))
    cur2_localmax.insert(i-samplesize, float(max(cur2_col1[i-samplesize:i-1])))
    #cur1_localmin.insert(i-samplesize, float(min(cur1_col1[i-samplesize:i-1])))
    cur2_localmin.insert(i-samplesize, float(min(cur2_col1[i-samplesize:i-1])))
    
    # Array boundary precausions
    if i > 0:
        o_r = (cur2_col1[i] - cur2_localmin[i-samplesize-1])/(cur2_localmax[i-samplesize-1] - cur2_localmin[i-samplesize-1])
        # Prevent o_r != {[0,1]}
        if o_r > 1:
            other_ratio.insert(i-samplesize, 1)
        elif o_r < 0:
            other_ratio.insert(i-samplesize, 0)
        else:
            other_ratio.insert(i-samplesize, o_r)

In [ ]:
# Preparing to plot on multiple y-axes
x = range(len(other_ratio))
fig = plt.figure(figsize=(150, 15))
ax1 = fig.add_subplot(111)

# Secondary data
ax1.plot(other_ratio, '#cdcdcd')
ax1.plot([1]*len(other_ratio), '#cdcdcd', linewidth=1.0)
ax1.plot([0]*len(other_ratio), '#cdcdcd', linewidth=1.0)
ax1.set_ylabel('ratio')
ax1.locator_params(nbins=30)

ax2 = ax1.twinx()
ax2.plot(cur2_localmax, 'r', linewidth=1.0)
ax2.plot(cur2_localmin, 'r', linewidth=1.0)
ax2.plot(cur2_col1[samplesize:len(other_ratio)+samplesize], 'r', linewidth=1.0)
ax2.locator_params(nbins=30)
ax2.set_ylabel('price', color='r')
for tl in ax2.get_yticklabels():
    tl.set_color('r')

plt.title('EURCAD period=' + str(samplesize))
plt.xlabel('(ticks)')
#plt.show()
plt.savefig('EURCAD_coint.pdf')